In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

#Ignore warnings for now 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Models
from sktime.classification.interval_based import CanonicalIntervalForest
from sktime.classification.interval_based import DrCIF
from sktime.classification.kernel_based import RocketClassifier

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [3]:
folder = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Data/freq_6_second_files/'

dtypes =  ['Wake', 'N1', 'N2', 'N3','REM']
results_dict = {}

for data_type in dtypes:
    results_dict[data_type] = {}
    results_dict[data_type]['X'] = pd.read_hdf(folder + data_type + 'six_second_freq_df.h5' , key = 'df', mode = 'r')
    results_dict[data_type]['groups'] = pd.read_hdf(folder + data_type + '_groups.h5' , key = 'df', mode = 'r')
    results_dict[data_type]['y'] = pd.read_hdf(folder + data_type + '_y.h5' , key = 'df', mode = 'r')

In [9]:
X = results_dict['Wake']['X'].copy()
y = results_dict['Wake']['y'].copy()
groups = results_dict['Wake']['groups'].copy()

X_sub = X.iloc[:30 , :2]
y_sub = y.iloc[:30]

In [11]:
clf =  RocketClassifier(use_multivariate='yes')

In [18]:
clf.fit(X_sub, y_sub)

ValueError: Data seen by RocketClassifier instance has unequal length series, but this RocketClassifier instance cannot handle unequal length series. Calls with unequal length series may result in error or unreliable results.